In [1]:
from __future__ import division, print_function
import sys, os, glob, time, warnings, gc
import numpy as np
import matplotlib.pyplot as plt
from astropy.table import Table, vstack, hstack
import fitsio
# from astropy.io import fits

In [2]:
params = {'legend.fontsize': 'large',
         'axes.labelsize': 'large',
         'axes.titlesize':'large',
         'xtick.labelsize':'large',
         'ytick.labelsize':'large',
         'figure.facecolor':'w'} 
plt.rcParams.update(params)

In [3]:
def create_image(data, cmap='gray', dpi=80, vmin=None, vmax=None, origin=None, norm=None):
    '''
    Create an image with exactly the same pixel dimension as the data.
    Example:
        x, y = np.arange(0, 10), np.arange(0, 10)
        xx, yy = np.meshgrid(x, y)
        img = np.array((xx + yy)%2==0, dtype=int)
        ax = create_image(img)
        plt.savefig('img.png')
        plt.close()
    '''
    xpixels, ypixels = data.shape[0], data.shape[1]
    figsize = ypixels / dpi, xpixels / dpi
    fig = plt.figure(figsize=figsize, dpi=dpi)
    ax = fig.add_axes([0, 0, 1, 1])
    ax.imshow(data, cmap=cmap, interpolation='none', vmin=vmin, vmax=vmax, origin=origin, norm=norm)
    plt.axis('off')
    return ax

In [4]:
bits_to_mask = [0, 2, 3, 4, 5, 6, 7]

In [10]:
brickname = '1286p200'
band = 'r'

img = fitsio.read('/global/cfs/cdirs/cosmo/work/legacysurvey/dr9m/south/coadd/{}/{}/legacysurvey-{}-image-{}.fits.fz'.format(brickname[:3], brickname, brickname, band))
ivar = fitsio.read('/global/cfs/cdirs/cosmo/work/legacysurvey/dr9m/south/coadd/{}/{}/legacysurvey-{}-invvar-{}.fits.fz'.format(brickname[:3], brickname, brickname, band))
nexp = fitsio.read('/global/cfs/cdirs/cosmo/work/legacysurvey/dr9m/south/coadd/{}/{}/legacysurvey-{}-nexp-{}.fits.fz'.format(brickname[:3], brickname, brickname, band))
maskbits = fitsio.read('/global/cfs/cdirs/cosmo/work/legacysurvey/dr9m/south/coadd/{}/{}/legacysurvey-{}-maskbits.fits.fz'.format(brickname[:3], brickname, brickname))

npix_obs = np.sum(nexp>0)

mask_clean = np.ones(img.shape, dtype=bool)
for bit in bits_to_mask:
    mask_clean &= (maskbits & 2**bit)==0

# Case 1
mask = (img==0) & (nexp>0) & (ivar>0)
mask &= mask_clean
npix_case1 = np.sum(mask)

# Case 2
mask = (np.abs(img)<1e-4) & (nexp>0) & (ivar>0)
mask &= mask_clean

# downsize to remove isolated pixels
binsize = 3
trim_size_x = mask.shape[1] % binsize
trim_size_y = mask.shape[0] % binsize
mask = mask[:(mask.shape[0]-trim_size_y), :(mask.shape[1]-trim_size_x)]
# to ignore NAN values, use np.nanmean
mask = np.median(np.median(mask.reshape((mask.shape[0]//binsize, binsize, mask.shape[1]//binsize,-1)), axis=3), axis=1)
mask = mask>0

npix_case2 = np.sum(mask)

# Case 3
mask = (nexp==0) & (ivar==0) & (img!=0)
mask &= mask_clean
npix_case3 = np.sum(mask)

print([npix_obs, npix_case1, npix_case2, npix_case3])

[12955448, 127560, 20167, 0]


In [11]:
brickname = '3196p182'
band = 'r'

img = fitsio.read('/global/cfs/cdirs/cosmo/work/legacysurvey/dr9m/south/coadd/{}/{}/legacysurvey-{}-image-{}.fits.fz'.format(brickname[:3], brickname, brickname, band))
ivar = fitsio.read('/global/cfs/cdirs/cosmo/work/legacysurvey/dr9m/south/coadd/{}/{}/legacysurvey-{}-invvar-{}.fits.fz'.format(brickname[:3], brickname, brickname, band))
nexp = fitsio.read('/global/cfs/cdirs/cosmo/work/legacysurvey/dr9m/south/coadd/{}/{}/legacysurvey-{}-nexp-{}.fits.fz'.format(brickname[:3], brickname, brickname, band))
maskbits = fitsio.read('/global/cfs/cdirs/cosmo/work/legacysurvey/dr9m/south/coadd/{}/{}/legacysurvey-{}-maskbits.fits.fz'.format(brickname[:3], brickname, brickname))

npix_obs = np.sum(nexp>0)

mask_clean = np.ones(img.shape, dtype=bool)
for bit in bits_to_mask:
    mask_clean &= (maskbits & 2**bit)==0

# Case 1
mask = (img==0) & (nexp>0) & (ivar>0)
mask &= mask_clean
npix_case1 = np.sum(mask)

# Case 2
mask = (np.abs(img)<1e-4) & (nexp>0) & (ivar>0)
mask &= mask_clean

# downsize to remove isolated pixels
binsize = 3
trim_size_x = mask.shape[1] % binsize
trim_size_y = mask.shape[0] % binsize
mask = mask[:(mask.shape[0]-trim_size_y), :(mask.shape[1]-trim_size_x)]
# to ignore NAN values, use np.nanmean
mask = np.median(np.median(mask.reshape((mask.shape[0]//binsize, binsize, mask.shape[1]//binsize,-1)), axis=3), axis=1)
mask = mask>0

npix_case2 = np.sum(mask)

# Case 3
mask = (nexp==0) & (ivar==0) & (img!=0)
mask &= mask_clean
npix_case3 = np.sum(mask)

print([npix_obs, npix_case1, npix_case2, npix_case3])

[12729326, 0, 164247, 0]


In [12]:
brickname = '3580p330'
band = 'r'

img = fitsio.read('/global/cfs/cdirs/cosmo/work/legacysurvey/dr9m/south/coadd/{}/{}/legacysurvey-{}-image-{}.fits.fz'.format(brickname[:3], brickname, brickname, band))
ivar = fitsio.read('/global/cfs/cdirs/cosmo/work/legacysurvey/dr9m/south/coadd/{}/{}/legacysurvey-{}-invvar-{}.fits.fz'.format(brickname[:3], brickname, brickname, band))
nexp = fitsio.read('/global/cfs/cdirs/cosmo/work/legacysurvey/dr9m/south/coadd/{}/{}/legacysurvey-{}-nexp-{}.fits.fz'.format(brickname[:3], brickname, brickname, band))
maskbits = fitsio.read('/global/cfs/cdirs/cosmo/work/legacysurvey/dr9m/south/coadd/{}/{}/legacysurvey-{}-maskbits.fits.fz'.format(brickname[:3], brickname, brickname))

npix_obs = np.sum(nexp>0)

mask_clean = np.ones(img.shape, dtype=bool)
for bit in bits_to_mask:
    mask_clean &= (maskbits & 2**bit)==0

# Case 1
mask = (img==0) & (nexp>0) & (ivar>0)
mask &= mask_clean
npix_case1 = np.sum(mask)

# Case 2
mask = (np.abs(img)<1e-4) & (nexp>0) & (ivar>0)
mask &= mask_clean

# downsize to remove isolated pixels
binsize = 3
trim_size_x = mask.shape[1] % binsize
trim_size_y = mask.shape[0] % binsize
mask = mask[:(mask.shape[0]-trim_size_y), :(mask.shape[1]-trim_size_x)]
# to ignore NAN values, use np.nanmean
mask = np.median(np.median(mask.reshape((mask.shape[0]//binsize, binsize, mask.shape[1]//binsize,-1)), axis=3), axis=1)
mask = mask>0

npix_case2 = np.sum(mask)

# Case 3
mask = (nexp==0) & (ivar==0) & (img!=0)
mask &= mask_clean
npix_case3 = np.sum(mask)

print([npix_obs, npix_case1, npix_case2, npix_case3])

[12733131, 0, 3, 215418]


In [13]:
# Normal brick
brickname = '1129p170'
band = 'r'

img = fitsio.read('/global/cfs/cdirs/cosmo/work/legacysurvey/dr9m/south/coadd/{}/{}/legacysurvey-{}-image-{}.fits.fz'.format(brickname[:3], brickname, brickname, band))
ivar = fitsio.read('/global/cfs/cdirs/cosmo/work/legacysurvey/dr9m/south/coadd/{}/{}/legacysurvey-{}-invvar-{}.fits.fz'.format(brickname[:3], brickname, brickname, band))
nexp = fitsio.read('/global/cfs/cdirs/cosmo/work/legacysurvey/dr9m/south/coadd/{}/{}/legacysurvey-{}-nexp-{}.fits.fz'.format(brickname[:3], brickname, brickname, band))
maskbits = fitsio.read('/global/cfs/cdirs/cosmo/work/legacysurvey/dr9m/south/coadd/{}/{}/legacysurvey-{}-maskbits.fits.fz'.format(brickname[:3], brickname, brickname))

npix_obs = np.sum(nexp>0)

mask_clean = np.ones(img.shape, dtype=bool)
for bit in bits_to_mask:
    mask_clean &= (maskbits & 2**bit)==0

# Case 1
mask = (img==0) & (nexp>0) & (ivar>0)
mask &= mask_clean
npix_case1 = np.sum(mask)

# Case 2
mask = (np.abs(img)<1e-4) & (nexp>0) & (ivar>0)
mask &= mask_clean

# downsize to remove isolated pixels
binsize = 3
trim_size_x = mask.shape[1] % binsize
trim_size_y = mask.shape[0] % binsize
mask = mask[:(mask.shape[0]-trim_size_y), :(mask.shape[1]-trim_size_x)]
# to ignore NAN values, use np.nanmean
mask = np.median(np.median(mask.reshape((mask.shape[0]//binsize, binsize, mask.shape[1]//binsize,-1)), axis=3), axis=1)
mask = mask>0

npix_case2 = np.sum(mask)

# Case 3
mask = (nexp==0) & (ivar==0) & (img!=0)
mask &= mask_clean
npix_case3 = np.sum(mask)

print([npix_obs, npix_case1, npix_case2, npix_case3])

[8880532, 0, 1, 2]
